In [1]:
ds_dir="/home/apsisdev/ansary/DATASETS/APSIS/Recognition/datasets/"
bn_datasets=["bangla_printed","bangla_handwritten","bs","bw","bh","iit.bn"]
en_datasets=["en_hw","english_printed","icdar13"]
# imports
import sys
sys.path.append('../')

import os 
from glob import glob
from tqdm.auto import tqdm
from coreLib.processing import processImages
#processImages(df,img_dim,ptype="left",factor=32)

In [5]:
# process icdar images
icdar_images=[img_path for img_path in tqdm(glob(os.path.join(ds_dir,"icdar13","images","*.*")))]
import pandas as pd
tqdm.pandas()
df=pd.DataFrame({"filepath":icdar_images})
df

  0%|          | 0/1095 [00:00<?, ?it/s]

,filepath
0,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...
1,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...
2,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...
3,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...
4,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...
...,...
1090,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...
1091,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...
1092,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...
1093,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...


In [7]:
df=processImages(df,img_dim=(64,512),ptype="left",factor=32)
df

100%|██████████| 1095/1095 [00:02<00:00, 545.60it/s]


,filepath,mask
0,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...,"[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, ..."
3,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...,"[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
1090,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...,"[1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1091,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...,"[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1092,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...,"[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1093,/home/apsisdev/ansary/DATASETS/APSIS/Recogniti...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, ..."


In [19]:
# classification
def get_label(img_path):
    # bangla,english
    dsname=os.path.basename(os.path.dirname(os.path.dirname(img_path)))
    if dsname in en_datasets:
        return [0,1]
    else:
        return [1,0]

In [13]:
eng_imgs=[]
ban_imgs=[]
for dataset in en_datasets:
    eng_imgs+=[img_path for img_path in tqdm(glob(os.path.join(ds_dir,dataset,"images","*.*")))]
print(len(eng_imgs))
for dataset in bn_datasets:
    ban_imgs+=[img_path for img_path in tqdm(glob(os.path.join(ds_dir,dataset,"images","*.*")))]
print(len(ban_imgs))


  0%|          | 0/42561 [00:00<?, ?it/s]

  0%|          | 0/998757 [00:00<?, ?it/s]

  0%|          | 0/1095 [00:00<?, ?it/s]

1042413


  0%|          | 0/999900 [00:00<?, ?it/s]

...

  0%|          | 0/59704 [00:00<?, ?it/s]

  0%|          | 0/21221 [00:00<?, ?it/s]

  0%|          | 0/107214 [00:00<?, ?it/s]

  0%|          | 0/113075 [00:00<?, ?it/s]

1401114


In [15]:
split=0.8
data_len=int(min(len(eng_imgs),len(ban_imgs))*split)
data_len

833930

In [16]:
import random
random.shuffle(eng_imgs)
random.shuffle(ban_imgs)
train_imgs=ban_imgs[:data_len]+eng_imgs[:data_len]
test_imgs =ban_imgs[data_len:]+eng_imgs[data_len:]

In [17]:
print(len(test_imgs))
print(len(train_imgs))

775667
1667860


In [22]:
data_size=10240
import tensorflow as tf
from coreLib.utils import create_dir
# feature fuctions
def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))
def _int64_list_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=value))


def toTfrecord(paths,rnum,rec_path):
    '''
        args:
            df      :   the dataframe that contains the information to store
            rnum    :   record number
            rec_path:   save_path
            mask_dim:   the dimension of the mask
    '''
    tfrecord_name=f'{rnum}.tfrecord'
    tfrecord_path=os.path.join(rec_path,tfrecord_name) 
    with tf.io.TFRecordWriter(tfrecord_path) as writer:    
        
        for img_path in paths:
            # img
            with(open(img_path,'rb')) as fid:
                image_png_bytes=fid.read()
            # feature desc
            data ={ 'image':_bytes_feature(image_png_bytes)}

            data["label"]=_int64_list_feature(get_label(img_path)) 

            
            features=tf.train.Features(feature=data)
            example= tf.train.Example(features=features)
            serialized=example.SerializeToString()
            writer.write(serialized)  

def createRecords(data,save_path,tf_size=10240):
    for idx in tqdm(range(0,len(data),tf_size)):
        paths        =   data[idx:idx+tf_size]  
        rnum      =   idx//tf_size
        toTfrecord(paths,rnum,save_path)

In [ ]:
random.shuffle(train_imgs)
random.shuffle(test_imgs)
rec_dir=create_dir(ds_dir,"classification")
train_save=create_dir(rec_dir,"train")
test_save=create_dir(rec_dir,"test")
createRecords(train_imgs,train_save)
createRecords(test_imgs,test_save)


  0%|          | 0/163 [00:00<?, ?it/s]

  0%|          | 0/76 [00:00<?, ?it/s]